# Running HR Model Live Inference on Edge TPU

The following set of codes is for executing the inferencing in loop based on video feed 
from the camera. <br>Camera snapshots are taken after every specific time interval and 
the raw image is processed and fed as  <br> input to the model for inferenceing.

Note on Usage:
This file is to be executed on an Edge TPU, typically the Coral Dev Board. A jupyter notebook has been<br> 
installed on the Coral Board under user 'Mendel'. Type 'jupyter notebook' or 'jupyter lab' in the linux<br>
command prompt to open the code files.

### Importing necessary dependencies, including Pycoral API

For this the pycoral API has to be installed. <link>https://coral.ai/docs/reference/py/</link> <br> 
Pycoral API and the necessary dependencies are already installed on the Coral Board ("tuned shrimp") <br> 
under the user 'Mendel'.

VideoStream library is used from 'imutils' to display the live camera feed. Use 'pip install imutils' <br> 
for using the VideoStream. Alternatively, OpenCV.VideoCapture() can also be used.

In [ ]:
import os
import time
import cv2
import matplotlib.pyplot as plt
from IPython.display import display, Image
from IPython.display import clear_output
#import ipywidgets as widget
import threading
import numpy as np
from pycoral.utils import edgetpu
from pycoral.utils import dataset
from pycoral.adapters import common
from pycoral.adapters import classify
from PIL import Image, ImageOps
import imutils
from imutils.video import VideoStream

### Specifying the model, labels and images

Here the model specified has to be compatible with Edge TPU. Ensure the model is converted, quantized <br>
and compiled with the Edgetpu compiler. The model file name should have 'edgetpu.tflite' in the end.

In [ ]:
model_file = 'hr_model_quant_edgetpu.tflite' # model edgetpu-tflite file
label_file = 'digits_labels.txt' # file with labels for classes
image_file = '2_img.GIF' # chose your image file here for inference

### Building the interpreter

In [ ]:
# Initialize the TF interpreter
interpreter = edgetpu.make_interpreter(model_file)
interpreter.allocate_tensors()
size = common.input_size(interpreter)
#run the below code to extract labels
# labels = dataset.read_label_file(label_file)

In [ ]:
# Checks if the model is integer quantized or not
if common.input_details(interpreter, 'dtype') != np.uint8:
    raise ValueError('Only support uint8 input type.')
else:
    print('model is quantized, ready to use')

### Initiating the VideoStream

In [ ]:
# initialise video streamer
vs = VideoStream(src=1).start() # change the source according to which camera you are using
print("starting video stream...")
time.sleep(2.0)

## Running the Inference in Loop

Note: the inference model yields good accuracy only if the captured images have minimal noises and <br> high contrast. For this the handwriting figures should be in dark color and on a plane white paper or background. <br> There should be enough brigh light and no shadows cast on the figures if possible.

In [ ]:
while True:
    # take the frame from the threaded video stream and resize to maximum width of 500 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=500)
    
    # prepare the frame for classification by converting it from
    # BGR to Gray channel ordering and then from a NumPy array to PIL image format
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # convert the image to binary with threshold value of 120 for easier inference
    th, im_th = cv2.threshold(frame, 120, 255, cv2.THRESH_BINARY)
    
    img = (Image.fromarray(im_th))
    # convert the image to numpy array
    
    frame_arr = np.array(img.resize(size, Image.ANTIALIAS))
    # broadcasting frame_img to fit dimentions (28, 28, 1)
    frame_arr_trfm = frame_arr.reshape((28,28,1))
    
    # execute the inference
    common.set_input(interpreter, frame_arr_trfm)
    interpreter.invoke()
    classes = classify.get_classes(interpreter, top_k=1)
    # display stream and print result
    plt.imshow(frame) # original captured frame
    plt.show()
    plt.imshow(im_th, cmap='gray') # binary image
    plt.show()
    print(classes)
    plt.imshow(frame_arr) # final resized image
    plt.show() 

    time.sleep(1) # delay
    clear_output(wait=True)
    
cv.destroyAllWindows()
vs.stop()